In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import f1_score
from tensorflow.keras.models import Model

In [3]:
# Set the image and model parameters
image_width, image_height = 128, 128
num_classes = 2
batch_size = 16
epochs = 10

In [4]:
# Paths to your data directories
image_dir = r"C:\Users\MSD\Documents\Datasets\MonReader\images\images"
training_dir = os.path.join(image_dir, 'training')
testing_dir = os.path.join(image_dir, 'testing')

In [5]:
# Load and preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(image_width, image_height))
    img = img_to_array(img) / 255.0  # Normalize pixel values
    return img

def load_images_from_directory(directory):
    images = []
    labels = []
    for label_dir in ['flip', 'notflip']:
        class_dir = os.path.join(directory, label_dir)
        class_label = 1 if label_dir == 'flip' else 0
        if not os.path.exists(class_dir):
            print(f"Directory does not exist: {class_dir}")
            continue
        for file in os.listdir(class_dir):
            if file.endswith('.jpg') or file.endswith('.png'):
                img_path = os.path.join(class_dir, file)
                img = preprocess_image(img_path)
                images.append(img)
                labels.append(class_label)
    return np.array(images), np.array(labels)

In [6]:
# Load datasets
X_train, y_train = load_images_from_directory(training_dir)
X_test, y_test = load_images_from_directory(testing_dir)

In [7]:
# Convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [8]:
# Split training set to include a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
# Building the CNN model
inputs = tf.keras.Input(shape=(image_width, image_height, 3))
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5560 - loss: 0.6866 - val_accuracy: 0.8058 - val_loss: 0.4448
Epoch 2/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8407 - loss: 0.3941 - val_accuracy: 0.9165 - val_loss: 0.2234
Epoch 3/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9250 - loss: 0.1755 - val_accuracy: 0.9436 - val_loss: 0.1779
Epoch 4/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9717 - loss: 0.1014 - val_accuracy: 0.9854 - val_loss: 0.0770
Epoch 5/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9658 - loss: 0.0813 - val_accuracy: 0.9645 - val_loss: 0.0989
Epoch 6/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9849 - loss: 0.0458 - val_accuracy: 0.9687 - val_loss: 0.0780
Epoch 7/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9885 - loss: 0.0285 - val_accuracy: 0.9186 - val_loss: 0.2473
Epoch 8/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9867 - loss: 0.0447 - val_accu

In [12]:
# Evaluate the model
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)
y_test = np.argmax(y_test, axis=1)
f1 = f1_score(y_test, predictions, average='micro')
print("F1 Score:", f1)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
F1 Score: 0.9715242881072027


In [13]:
# Save the model
model.save('monreader_page_flip_detection_model.h5')